# Web Automation and Search information with Python

### Problem:

We working in an Importation, and our prices charged for our products are linked to the quotation:

- **Dollar**
- **Euro**
- **Gold**

We need to automatically get the price of these 3 items from internet and know how much we should charge for our products, considering a contribution margin that we have in our database.

DataBase: 

For this, we'll create a web automation:

- We'll use Selenium
- **IMPORTANT**: Download webdriver

In [11]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import packages

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [13]:
browser = webdriver.Chrome()

# step 1: get the Dollar price.

browser.get("https://www.google.com/")

browser.find_element(By.XPATH,
                     '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Dolar")

browser.find_element(By.XPATH,
                     '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

price_dollar = browser.find_element(By.XPATH,
                     '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(price_dollar)

5.5443999999999996


In [15]:
# step 2: get the Euro price.

browser.get("https://www.google.com/")

browser.find_element(By.XPATH,
                     '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Euro")

browser.find_element(By.XPATH,
                     '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

price_euro = browser.find_element(By.XPATH,
                     '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(price_euro)

# step 3: get the gold price.

browser.get("https://www.melhorcambio.com/ouro-hoje")

price_gold = browser.find_element(By.XPATH,
                     '//*[@id="comercial"]').get_attribute("value")
price_gold = price_gold.replace(",",".")

print(price_gold)

browser.quit()

6.349187314000001
325.53


## Now, let's update our Pricing Base with the new quotes.

- Import Database.

In [ ]:
# step 4: Import database and update the price in my database

import pandas as pd

table = pd.read_excel("C:/Users/Pedro/Downloads/Produtos.xlsx")
display(table)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Update prices and the final price calculation

In [ ]:
# Step 5: Calculate the new prices and save the database

table.loc[table['Moeda'] == 'Dólar','Cotação'] = float(price_dollar)
table.loc[table['Moeda'] == 'Euro','Cotação'] = float(price_euro)
table.loc[table['Moeda'] == 'Ouro','Cotação'] = float(price_gold)

display(table)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.538400,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.334804,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.538400,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.538400,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.334804,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.538400,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,324.290000,7000.00,1.15,8050.000


## Update columns


### - purchase price = Original Price * Quotes
### - Sales price = purchase Price * markup

In [ ]:
table["Preço de Compra"] = table["Preço Original"] * table["Cotação"]
table["Preço de Venda"] = table["Preço de Compra"] * table["Margem"]

display(table)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.538400,5538.344616,1.40,7753.682462
1,Carro Renault,4500.00,Euro,6.334804,28506.619980,2.00,57013.239960
2,Notebook Dell,899.99,Dólar,5.538400,4984.504616,1.70,8473.657847
3,IPhone,799.00,Dólar,5.538400,4425.181600,1.70,7522.808720
4,Carro Fiat,3000.00,Euro,6.334804,19004.413320,1.90,36108.385308
5,Celular Xiaomi,480.48,Dólar,5.538400,2661.090432,2.00,5322.180864
6,Joia 20g,20.00,Ouro,324.290000,6485.800000,1.15,7458.670000


### - Now we are going to export the new updated price base.

In [ ]:
table.to_excel("New_Products.xlsx", index=False)